In [2]:
pip install numpy tensorflow matplotlib keras

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import io
from PIL import Image

# Path to your datasets (adjust as needed)
data_dir = 'dataset'  # Ensure it contains subfolders for each class (e.g., 'Brain Tumor', 'Alzheimers', etc.)

# Image size and batch settings
img_height, img_width = 224, 224
batch_size = 32
epochs = 5  # Reduced epochs for quicker training

# Create ImageDataGenerator instances for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% validation split
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Use 'training' subset
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Use 'validation' subset
)

# Get the number of classes (diseases)
num_classes = len(train_generator.class_indices)

# Define multiple CNN models (Ensemble)
def build_model_1():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model

def build_model_2():
    model = Sequential()
    model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model

def build_model_3():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model

# Compile the models
def compile_model(model):
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Create three models for the ensemble
model_1 = build_model_1()
model_2 = build_model_2()
model_3 = build_model_3()

compile_model(model_1)
compile_model(model_2)
compile_model(model_3)

# Callbacks for early stopping and checkpointing
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
]

# Train all models individually
history_1 = model_1.fit(
    train_generator,
    steps_per_epoch=train_generator.samples / batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples / batch_size,
    epochs=epochs,
    callbacks=callbacks
)

history_2 = model_2.fit(
    train_generator,
    steps_per_epoch=train_generator.samples / batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples / batch_size,
    epochs=epochs,
    callbacks=callbacks
)

history_3 = model_3.fit(
    train_generator,
    steps_per_epoch=train_generator.samples / batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples / batch_size,
    epochs=epochs,
    callbacks=callbacks
)

# Combine the predictions of all models
def ensemble_predictions(models, test_generator):
    predictions = []
    for model in models:
        preds = model.predict(test_generator, steps=test_generator.samples // batch_size)
        predictions.append(preds)
    
    # Average the predictions
    avg_predictions = np.mean(predictions, axis=0)
    return avg_predictions

# Ensemble of the three models
models = [model_1, model_2, model_3]
ensemble_preds = ensemble_predictions(models, validation_generator)

# Save final ensemble model
for i, model in enumerate(models):
    model.save(f'ensemble_model_{i+1}.h5')

# Plot training and validation accuracy and loss for model_1 (repeat for other models if needed)
acc = history_1.history['accuracy']
val_acc = history_1.history['val_accuracy']
loss = history_1.history['loss']
val_loss = history_1.history['val_loss']
epochs_range = range(len(acc))  # Adjust based on actual epochs run

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy - Model 1')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss - Model 1')
plt.show()



In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

In [13]:
# Define constants
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 6
EPOCHS = 25

In [14]:
# Paths to training and validation datasets
train_dir = 'dataset/training'
val_dir = 'dataset/validation'

In [15]:
# Load the training and validation datasets
train_dataset = image_dataset_from_directory(
    train_dir,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical',  # One-hot encoded labels
    shuffle=True
)

Found 54320 files belonging to 6 classes.


In [16]:
validation_dataset = image_dataset_from_directory(
    val_dir,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical',  # One-hot encoded labels
)

Found 18022 files belonging to 6 classes.


In [17]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
])

In [18]:
# Model creation (simple CNN)
model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(224, 224, 3)),  # Normalize images
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation='softmax')  # 6 classes
])



In [19]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3)


In [20]:
# Train the model
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=5,
    callbacks=[early_stopping]
)


Epoch 1/5
1698/1698 [==============================] - 2455s 1s/step - loss: 0.1082 - accuracy: 0.9700 - val_loss: 0.0611 - val_accuracy: 0.9827
Epoch 2/5
1698/1698 [==============================] - 2528s 1s/step - loss: 0.0404 - accuracy: 0.9886 - val_loss: 0.1218 - val_accuracy: 0.9720
Epoch 3/5
1698/1698 [==============================] - 2456s 1s/step - loss: 0.0286 - accuracy: 0.9917 - val_loss: 0.0564 - val_accuracy: 0.9855
Epoch 4/5
1698/1698 [==============================] - 2387s 1s/step - loss: 0.0254 - accuracy: 0.9927 - val_loss: 0.0736 - val_accuracy: 0.9814
Epoch 5/5
1698/1698 [==============================] - 2390s 1s/step - loss: 0.0198 - accuracy: 0.9939 - val_loss: 0.0615 - val_accuracy: 0.9857


In [21]:
# Save the model
model.save('bdc.h5')

c:\Users\ar646\Desktop\Projects\BDD\pluto\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [22]:
# Evaluate the model on the validation dataset
val_loss, val_acc = model.evaluate(validation_dataset)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")


564/564 [==============================] - 141s 249ms/step - loss: 0.0615 - accuracy: 0.9857
Validation Accuracy: 98.57%
